In [ ]:
import os
source_dir = '.\\extracts'
target_dir = '.\\target'
all_files = []
for root, _, files in os.walk(source_dir):
        os.makedirs(root, exist_ok=True)
        for file in files:
            all_files.append(os.path.join(root, file))

print(os.path.splitdrive(os.path.abspath(target_dir)))

for file in all_files:
    print()
    print(file)
    print(os.path.dirname(file))
    print(os.path.basename(file))

    source_abs_path = os.path.abspath(file)
    src_rel_path = os.path.relpath(source_abs_path, source_dir)
    dest_rel_path = os.path.relpath(os.path.join(target_dir, src_rel_path))
    dest_abs_path = os.path.abspath(dest_rel_path)
    
    print(f"Source")
    print(f"abs: {source_abs_path}")
    print(f"rel: {src_rel_path}")

    print(f"Destination")
    print(f"rel: {dest_rel_path}")
    print(f"abs: {dest_abs_path}")


('c:', '\\rclone\\target')

.\src\sub1\another\test2.txt
.\src\sub1\another
test2.txt
Source
abs: c:\rclone\src\sub1\another\test2.txt
rel: sub1\another\test2.txt
Destination
rel: target\sub1\another\test2.txt
abs: c:\rclone\target\sub1\another\test2.txt

.\src\sub1\sub2\test.txt
.\src\sub1\sub2
test.txt
Source
abs: c:\rclone\src\sub1\sub2\test.txt
rel: sub1\sub2\test.txt
Destination
rel: target\sub1\sub2\test.txt
abs: c:\rclone\target\sub1\sub2\test.txt


In [ ]:
import shutil

source_dir = '.\\src'
operation = 'copy'  # or 'move'
target_dir = 'imported'
target_dir = os.path.join(os.path.expanduser('~\\Pictures'), target_dir)
is_same_drive = os.path.splitdrive(os.path.abspath(source_dir))[0].lower() == os.path.splitdrive(os.path.abspath(target_dir))[0].lower()

all_files = []
for root, _, files in os.walk(source_dir):
    for file in files:
        all_files.append(os.path.join(root, file))

target_dir = os.path.join(os.path.expanduser('~\\Pictures'), target_dir)
print(f"Target directory: {target_dir}")

# dest_file = os.path.join(target_dir, os.path.relpath(".\\src\\subfolder\\test.txt", start=source_dir))

for source_file in all_files:
    source_file = os.path.abspath(source_file)
    dest_file = os.path.join(target_dir, os.path.relpath(source_file, start=source_dir))
    print(f"Destination file: {dest_file}")
    if os.path.exists(dest_file):
        print(f"File already exists: {dest_file}. Skipping.")
        continue

    os.makedirs(os.path.dirname(dest_file), exist_ok=True)
    if operation == 'copy':
        shutil.copy2(source_file, dest_file)
        print(f"Copied: {source_file} to {dest_file}")
    else:  # move operation
        if is_same_drive:
            # On same drive, use rename (efficient move operation)
            os.rename(source_file, dest_file)
            print(f"Moved: {source_file} to {dest_file}")
        else:
            # Cross-drive move requires copy then delete
            shutil.copy2(source_file, dest_file)
            os.remove(source_file)
            print(f"Cross-drive move: {source_file} to {dest_file}")




Target directory: C:\Users\ddials\Pictures\imported
Destination file: C:\Users\ddials\Pictures\imported\sub1\another\test.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\another\test.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\another\test2.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\another\test2.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\sub2\test.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\sub2\test.txt. Skipping.
Destination file: C:\Users\ddials\Pictures\imported\sub1\sub2\test2.txt
File already exists: C:\Users\ddials\Pictures\imported\sub1\sub2\test2.txt. Skipping.


In [8]:
import os

source_dir = '.\\extracts\\Takeout\\Google Photos'
target_path = 'imported'
remote = 'onedrive'
rclone_path = 'rclone.exe'

try:
    # Build the rclone command with progress option
    cmd = [
        rclone_path, 
        "sync",  # Use 'sync' to mirror the source to the target
        # "--files-from", "upload_files_list.txt",
        "--progress",
        "--transfers", "4",  # Number of file transfers to run in parallel
        source_dir,
        f"{remote}:{target_path}"  # Destination
    ]
    
    print(f"Running command: {' '.join(cmd)}")
    import subprocess
    
    # Run the command and stream output in real-time
    process = subprocess.Popen(
        cmd,
        stdout=subprocess.PIPE,
        stderr=subprocess.STDOUT,
        universal_newlines=True,
        bufsize=1
    )
    
    # Process and log output in real-time
    for line in process.stdout:
        line = line.strip()
        if line:
            if "ERROR" in line:
                print(f"ERROR: {line}")
            elif "transferred:" in line or "Transferred:" in line:
                # Highlight progress info
                print(f"INFO: {line}")
            else:
                print(f"DEBUG: {line}")

    process.wait()
    
    if process.returncode == 0:
        print(f"✓ Upload completed successfully")
    else:
        print(f"✗ Upload failed with return code {process.returncode}")

except Exception as e:
    print(f"ERROR: Error during upload: {str(e)}")
# finally:
#     # Clean up temp file
#     if os.path.exists(temp_list_file):
#         os.remove(temp_list_file)
#         print("DEBUG: Removed temporary file list")


Running command: rclone.exe sync --progress --transfers 4 .\extracts\Takeout\Google Photos onedrive:imported
INFO: Transferred:   	          0 B / 1.906 MiB, 0%, 0 B/s, ETA -
INFO: Transferred:            0 / 5, 0%
DEBUG: Elapsed time:         0.4s
DEBUG: Transferring:
DEBUG: *                                 IMG_8823.HEIC:  0% /1.906Mi, 0/s, -
DEBUG: *                         sub1/another/test.txt:  0% /0, 0/s, -
DEBUG: *                            sub1/sub2/test.txt:  0% /0, 0/s, -
INFO: *                           sub1/sub2/test2.txt:  0% /0, 0/s, -Transferred:   	          0 B / 1.906 MiB, 0%, 0 B/s, ETA -
INFO: Transferred:            0 / 5, 0%
DEBUG: Elapsed time:         0.9s
DEBUG: Transferring:
DEBUG: *                                 IMG_8823.HEIC:  0% /1.906Mi, 0/s, -
DEBUG: *                         sub1/another/test.txt:  0% /0, 0/s, -
DEBUG: *                            sub1/sub2/test.txt:  0% /0, 0/s, -
INFO: *                           sub1/sub2/test2.txt:  0% /0, 0/s, 

In [ ]:
import argparse
import os
import sys

source_dir = "d:\\test"
target_dir = "d:\\livefiles"

for root, dirs, files in os.walk(source_dir):
    for file in files:
        if file.lower().endswith('.mp4'):
            print (f"Processing file: {file}")
            heic_path = f"{os.path.splitext(file)[0]}.heic"
            print (f"looking for heic file: {heic_path}")

            #if the file exists, remove the mp4 file
            if os.path.exists(os.path.join(root, heic_path)):
                print (f"heic file exists")
                source_file = os.path.join(root, file)
                dest_file = os.path.join(target_dir, os.path.relpath(source_file, start=source_dir))
                os.makedirs(os.path.dirname(dest_file), exist_ok=True)
                print (f"renaming {source_file} to {dest_file}")
                os.rename(source_file, dest_file)
            else :
                print (f"heic file does not exist")




Processing file: IMG_4275.MP4
looking for heic file: IMG_4275.heic
heic file exists
renaming d:\test\Photos from 2025\IMG_4275.MP4 to d:\livefiles\Photos from 2025\IMG_4275.MP4
Processing file: IMG_8793.MP4
looking for heic file: IMG_8793.heic
heic file exists
renaming d:\test\Photos from 2025\IMG_8793.MP4 to d:\livefiles\Photos from 2025\IMG_8793.MP4
Processing file: IMG_8794.MP4
looking for heic file: IMG_8794.heic
heic file exists
renaming d:\test\Photos from 2025\IMG_8794.MP4 to d:\livefiles\Photos from 2025\IMG_8794.MP4
Processing file: IMG_8799.MP4
looking for heic file: IMG_8799.heic
heic file exists
renaming d:\test\Photos from 2025\IMG_8799.MP4 to d:\livefiles\Photos from 2025\IMG_8799.MP4
Processing file: IMG_8810.MP4
looking for heic file: IMG_8810.heic
heic file exists
renaming d:\test\Photos from 2025\IMG_8810.MP4 to d:\livefiles\Photos from 2025\IMG_8810.MP4
Processing file: IMG_8811.MP4
looking for heic file: IMG_8811.heic
heic file exists
renaming d:\test\Photos from 20

In [ ]:
import datetime
import os, subprocess

root_folder = '..\\test'

for subdir, _, files in os.walk(root_folder):
    print(f"Processing directory: {subdir}")        
    for file in files:
        try:
            date_taken = None
            file_date = os.path.getctime(os.path.join(subdir, file))
            file_date = datetime.datetime.fromtimestamp(file_date).strftime('%Y:%m:%d %H:%M:%S')
            #use exiftool to retrieve the date taken from the file metadata
            exiftool_cmd = [
                'exiftool',
                '-s',  # Short output
                '-DateTimeOriginal',  # Get the original date taken
                os.path.join(subdir, file)
            ]
            # print(f"Running command: {' '.join(exiftool_cmd)}")
            result = subprocess.run(exiftool_cmd, capture_output=True, text=True)
            if result.returncode == 0:
                date_taken = result.stdout.strip().split(': ')[1]

            if date_taken and date_taken.split()[0].strip() != file_date.split()[0].strip():
                exiftool_cmd = [
                    'exiftool',
                    '-overwrite_original',  # Don't create backup files
                    # '-preserve',           # Preserve file modification date/time
                    f'-DateTimeOriginal={date_taken}',
                    f'-CreateDate={date_taken}', 
                    f'-ModifyDate={date_taken}'
                ]
            
                # Add the file path at the end
                exiftool_cmd.append(os.path.join(subdir, file))
                
                # Log the command for debugging
                print(f"Running command: {' '.join(exiftool_cmd)}")
        except Exception as e:
                print(f"Failed to process {file}: {str(e)}")

In [ ]:
import pywintypes
import os, subprocess, datetime

target_dir = '..\\test'
dates = []

for subdir, _, files in os.walk(target_dir):
    for file in files:
        print(f"Processing file: {file} in {subdir}")
        
        file_date = os.path.getctime(os.path.join(subdir, file))
        file_date = datetime.datetime.fromtimestamp(file_date).strftime('%Y:%m:%d %H:%M:%S')
        print(f"File creation date: {file_date}")

        exiftool_cmd = [
            'exiftool',
            '-s',  # Short output
            '-DateTimeOriginal',  # Get the original date taken
            os.path.join(subdir, file)
        ]
        result = subprocess.run(exiftool_cmd, capture_output=True, text=True)
        results = result.stdout.strip()
        print(f"ExifTool output: {results}")
        image_date = results.split(': ')[1]
        print(f"Image date: {image_date}")

        try :
            try:
                image_date = datetime.datetime.strptime(image_date, '%Y:%m:%d %H:%M:%S')
                image_date = image_date.replace(year=1971) if image_date.year < 1971 else image_date 
                print(f"Parsed image date for {file}: {image_date}")
            except ValueError as e:
                print(f"Using default date. Failed to parse image date for {file}: [{e}]")
        except ValueError:
            print(f"Invalid date format for {file}: {image_date} : {results}")
            
        dates.append(image_date)
        print(f"Updating {file} creation date from {file_date} to {image_date}")
        win_time = pywintypes.Time(image_date)
        print(f"Setting creation date to {win_time}")

print (f"All dates processed: {dates}")


Processing file: AuntRueyetal.jpg in ..\test
File creation date: 2025:06:05 11:31:54
ExifTool output: DateTimeOriginal                : 1970:01:01 03:00:00
Image date: 1970:01:01 03:00:00
Parsed image date for AuntRueyetal.jpg: 1970-01-01 01:00:01
Updating AuntRueyetal.jpg creation date from 2025:06:05 11:31:54 to 1970-01-01 01:00:01
Setting creation date to 1970-01-01 01:00:01
Processing file: DiningRoom.jpg in ..\test
File creation date: 2000:02:05 21:01:01
ExifTool output: DateTimeOriginal                : 2000:02:05 21:01:21
Image date: 2000:02:05 21:01:21
Parsed image date for DiningRoom.jpg: 2000-02-05 01:01:01
Updating DiningRoom.jpg creation date from 2000:02:05 21:01:01 to 2000-02-05 01:01:01
Setting creation date to 2000-02-05 01:01:01
All dates processed: [datetime.datetime(1970, 1, 1, 1, 0, 1), datetime.datetime(2000, 2, 5, 1, 1, 1)]
